### Example of FactReasoner Pipeline

In [ ]:
import os
from pathlib import Path
import sys
import json

parent_dir = str(Path(os.getcwd()).resolve().parent)
sys.path.insert(0, parent_dir)
print(parent_dir)

from src.fact_reasoner.context_retriever import ContextRetriever
from src.fact_reasoner.atom_extractor import AtomExtractor
from src.fact_reasoner.atom_reviser import AtomReviser
from src.fact_reasoner.query_builder import QueryBuilder
from src.fact_reasoner.context_summarizer import ContextSummarizer
from src.fact_reasoner.nli_extractor import NLIExtractor
from src.fact_reasoner.factreasoner import FactReasoner

In [ ]:
# Define the model and backend
model_id = "llama-3.3-70b-instruct"
cache_dir = None # "my_database.db"
backend = "rits"

# Define the input file and merlin path (change to ...)
input_file = "/home/radu/git/FactReasoner/examples/flaherty_wikipedia.json"
merlin_path = "/home/radu/git/fm-factual/lib/merlin"

# Create the pipeline modules
query_builder = QueryBuilder(model_id=model_id, prompt_version="v1", backend=backend)
context_retriever = ContextRetriever(service_type="google", top_k=5, cache_dir=cache_dir)
context_summarizer = ContextSummarizer(model_id=model_id, prompt_version="v1", backend=backend)
atom_extractor = AtomExtractor(model_id=model_id, backend=backend)
atom_reviser = AtomReviser(model_id=model_id, backend=backend)
nli_extractor = NLIExtractor(model_id=model_id, prompt_version="v1", backend=backend)

# Create the FactReasoner pipeline
pipeline = FactReasoner(
    context_retriever=context_retriever,
    context_summarizer=context_summarizer,
    atom_extractor=atom_extractor,
    atom_reviser=atom_reviser,
    nli_extractor=nli_extractor,
    query_builder=query_builder,
    merlin_path=merlin_path,
)

# Load the problem instance from a file
json_file = input_file
with open(json_file, "r") as f:
    data = json.load(f)

# Instantiate the pipeline from a data file
pipeline.from_dict_with_contexts(data)

# Build the FactReasoner pipeline (FR2 version)
pipeline.build(
    has_atoms=True,
    has_contexts=True,
    revise_atoms=False,
    remove_duplicates=True,
    contexts_per_atom_only=False,
    rel_atom_context=True, 
    rel_context_context=False,
    text_only=False
)

# Compute the marginals
results, marginals = pipeline.score()
print(f"[FactReasoner] Marginals: {marginals}")
print(f"[FactReasoner] Results: {results}")
print(f"Done.")